In [1]:
%gui qt

In [2]:
from importlib import reload

import numpy as np

import pyqtgraph as pg
from pyqtgraph.flowchart import Flowchart, Node as pgNode
from pyqtgraph.Qt import QtGui, QtCore
from pyqtgraph.flowchart import library as fclib

from plottr.data import datadict; reload(datadict)
from plottr.data.datadict import togrid, DataDict, GridDataDict

In [11]:
def testdata_1d(nvals=11):
    x = np.linspace(0, 10, nvals)
    y = np.cos(x)
    z = np.cos(x)**2
    d = DataDict(
        x = {'values' : x},
        y = {'values' : y, 'axes' : ['x']},
        z = {'values' : z, 'axes' : ['x']},
    )
    return d

def testdata_3d(nx=3, ny=3, nz=3):
    x = np.linspace(0, 10, nx)
    y = np.linspace(-5, 5, ny)
    z = np.arange(nz)
    xx, yy, zz = np.meshgrid(x, y, z, indexing='ij')
    dd = np.cos(xx) * np.sin(yy) + np.random.rand(*zz.shape)
    dd2 = np.sin(xx) * np.cos(yy) + np.random.rand(*zz.shape)
    d = DataDict(
        x = dict(values=xx.reshape(-1)),
        y = dict(values=yy.reshape(-1)),
        z = dict(values=zz.reshape(-1)),
        data = dict(values=dd.reshape(-1), axes=['x', 'y', 'z']),
        more_data = dict(values=dd2.reshape(-1), axes=['x', 'y', 'z']),
        different_data = dict(values=dd2.T.reshape(-1), axes=['z', 'y', 'x'])
    )
    return d

In [26]:
class Node(pgNode):
    
    nodeName = "DataDictNode"
    terminals = {
        'dataIn' : {'io' : 'in'},
        'dataOut' : {'io' : 'out'},
    }
    
    def __init__(self, name):
        super().__init__(name, terminals=self.__class__.terminals)
        
        self.raiseExceptions = False
        self.signalUpdate = True
        
        self._grid = False
        self._gridData = None
        
    
    def updateOption(func):
        def wrap(self, val):
            ret = func(self, val)
            self.update(self.signalUpdate)
            return ret
        return wrap
    
    def update(self, signal=True):
        super().update(signal=signal)
        if self.raiseExceptions and self.exception is not None:
            raise self.exception[1]
    
    @property
    def grid(self):
        return self._grid

    @grid.setter
    @updateOption
    def grid(self, val):
        self._grid = val
    
    @property
    def gridData(self):
        return self._gridData

    @gridData.setter
    @updateOption
    def gridData(self, val):
        self._gridData = val
    
    def process(self, **kw):
        ret = kw['dataIn']
        if self.grid:
            ret = togrid(ret, names=self.gridData)
        
        return dict(dataOut=ret)
    

nodelib = fclib.NodeLibrary()
nodelib.addNodeType(Node, [('Basic')])

fc = Flowchart(terminals={
    'dataIn': {'io': 'in'},
    'dataOut': {'io': 'out'}
})
fc.library = nodelib
fc.setInput(dataIn=testdata_3d())

trivial_node = fc.createNode('DataDictNode')
trivial_node.raiseExceptions = True

fc.connectTerminals(fc['dataIn'], trivial_node['dataIn'])
fc.connectTerminals(trivial_node['dataOut'], fc['dataOut'])

In [27]:
fc.inputValues(), fc.output()

({'dataIn': None},
 {'dataOut': {'x': {'values': array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   5.,   5.,
             5.,   5.,   5.,   5.,   5.,   5.,   5.,  10.,  10.,  10.,  10.,
            10.,  10.,  10.,  10.,  10.])},
   'y': {'values': array([-5., -5., -5.,  0.,  0.,  0.,  5.,  5.,  5., -5., -5., -5.,  0.,
            0.,  0.,  5.,  5.,  5., -5., -5., -5.,  0.,  0.,  0.,  5.,  5.,  5.])},
   'z': {'values': array([0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1,
           2, 0, 1, 2])},
   'data': {'values': array([ 1.02338833,  1.56571974,  1.75158259,  0.65782687,  0.71702436,
            0.90367998, -0.45427166, -0.23960919, -0.61214794,  0.69415969,
            0.85981913,  0.72623975,  0.81038208,  0.6389959 ,  0.18651799,
            0.04449455,  0.35352561,  0.16705926, -0.04032021, -0.27134624,
            0.09068973,  0.28514704,  0.58970555,  0.02618285,  0.88710067,
            0.8198449 ,  0.80933304]), 'axes': ['x', 'y', 'z']}

In [34]:
trivial_node.signalUpdate = True
trivial_node.gridData = 'data', 'more_data'
trivial_node.grid = True

In [35]:
fc.inputValues(), fc.output()

({'dataIn': None},
 {'dataOut': {'x': {'values': array([  0.,   5.,  10.]),
    'unit': '',
    'axes': []},
   'y': {'values': array([-5.,  0.,  5.]), 'unit': '', 'axes': []},
   'z': {'values': array([0, 1, 2], dtype=int64), 'unit': '', 'axes': []},
   'data': {'values': array([[[ 1.02338833,  1.56571974,  1.75158259],
            [ 0.65782687,  0.71702436,  0.90367998],
            [-0.45427166, -0.23960919, -0.61214794]],
    
           [[ 0.69415969,  0.85981913,  0.72623975],
            [ 0.81038208,  0.6389959 ,  0.18651799],
            [ 0.04449455,  0.35352561,  0.16705926]],
    
           [[-0.04032021, -0.27134624,  0.09068973],
            [ 0.28514704,  0.58970555,  0.02618285],
            [ 0.88710067,  0.8198449 ,  0.80933304]]]),
    'axes': ['x', 'y', 'z'],
    'unit': ''},
   'more_data': {'values': array([[[ 0.37791773,  0.56100562,  0.37764528],
            [ 0.28336787,  0.51414885,  0.91337011],
            [ 0.76974657,  0.63832241,  0.64325889]],
    
    